In [ ]:
!pip install -r "requirements.txt"

### Read data

In [31]:
import pandas as pd
import numpy as np

In [4]:
path = "data/data.csv"
data = pd.read_csv(path, index_col=0)

### Processing

In [191]:
import re
from natasha import (
    Segmenter, MorphVocab,
    NewsEmbedding, NewsNERTagger,
    NewsMorphTagger, Doc
)
from difflib import get_close_matches
from city_data import cities, city_mapping

In [249]:
# Natasha компоненты
segmenter = Segmenter()
emb = NewsEmbedding()
ner_tagger = NewsNERTagger(emb)
morph_tagger = NewsMorphTagger(emb)
morph_vocab = MorphVocab()

In [250]:
def normalize_city_name(text):
    norm = text.lower().strip()
    if norm in city_mapping:
        return city_mapping[norm]
    return norm.capitalize()

def fuzzy_match_city(word):
    candidates = list(cities) + list(city_mapping.keys())
    match = get_close_matches(word, candidates, n=1, cutoff=0.8)
    if match:
        candidate = match[0]
        return city_mapping.get(candidate.lower(), candidate)
    return None

In [251]:
def extract_city(text):
    text_clean = text.lower().replace('.', '').replace('-', ' ')
    
    # 1. Поиск по city_mapping напрямую
    for alias, real in city_mapping.items():
        if re.search(rf'\\b{alias}\\b', text_clean):
            return real

    # 2. Морфология через Natasha
    doc = Doc(text)
    doc.segment(segmenter)
    doc.tag_morph(morph_tagger)
    doc.tag_ner(ner_tagger)

    for span in doc.spans:
        if span.type == 'LOC':
            span.normalize(morph_vocab)
            city = span.normal
            if city in cities:
                return city
            fuzzy = fuzzy_match_city(city.lower())
            if fuzzy:
                return fuzzy

    for token in doc.tokens:
        token.lemmatize(morph_vocab)
        lemma = token.lemma
        if lemma in cities:
            return lemma
        if lemma.lower() in city_mapping:
            return city_mapping[lemma.lower()]
        fuzzy = fuzzy_match_city(lemma.lower())
        if fuzzy:
            return fuzzy

    return None

#### Sample

In [306]:
sample = data.sample(ignore_index=True).loc[0, 'message']
print(sample)
print(extract_city(sample))

Добрый вечер. Хочу заказать букет роз для девушки в Мск. Можно добавить плюшевого мишку к заказу? Сколько это будет стоить? Спасибо!
Москва


### Test

In [298]:
data['city'] = data['message'].apply(extract_city)

In [301]:
data['city'].isnull().sum() # Нет пропусков

0

In [302]:
data.value_counts('city') # Отличное распределение городов

city
Новосибирск        25
Екатеринбург       24
Москва             22
Нижний Новгород    22
Санкт-Петербург    20
Алматы              3
Астана              2
Нур-Султан          2
Name: count, dtype: int64

### Save

In [ ]:
# Данное решение является воспроизводимым на любых data.csv схожих по формату с изначальными

In [308]:
data.to_csv(path)